In [23]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir) + "/src/"
sys.path.insert(0, parentdir) 

from modeling import get_effort_filter_value_options
from modelchecking import trialwise_rewards, trialwise_greedydiff, trialwise_chooseleft
from analysis import Analyzer, lmm, glmm
from plots import set_helvetica_style
from utils import colormaps, report_p_value, strsimplify, get_stochasticity_levels, alphabet, get_data, preprocess_data
import json
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Demo for Model Fitting
## Step 1: Data Loading

Let's try to fit some data and see how it goes. First, let's load the data for a stochasticity variant of our choice.

In [94]:
variant = "R"
data = get_data(variant = variant)

# get the first participant
participant = list(data.keys())[0]

data[participant] is a list of game data. We can take a look at one of these games.
- 'actions' contains a sequence of actions (True if left, False if right)
- 'baseline' contains the expected reward from a random policy on the boards
- 'boards' contains the face-value of the boards visible to the participant at every step (for example, in the volatility case this changes every step)
- 'p' contains the stochasticity level
- 'tuplepath' is the path the participant took, in a tuple
- 'total' is the total reward
- 'trials' contains the reaction times for each of the trials
- many other fields that aren't used much in the modeling part

In [95]:

print("Boards:\n", np.array(data[participant][0].boards)) # this is a list, but we just show the first board for now
print("Actions: ", data[participant][0].actions)
print("Tuplepath: ", data[participant][0].tuplepath)

stochasticity_level = data[participant][0].p
print("Stochasticity level: ", stochasticity_level)


Boards:
 [[[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 4 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]

We then preprocess the data from the participant. What we do is basically use the `xr.DataArray` object to combine these into a neat
multi-dimensional array that splits by conditions, games, trials, rows, columns. It basically just makes the data easier to organize, 
and makes it easier for us to read the data.

In [96]:
processed_data = preprocess_data(data[participant])

# so for example, the boards for the game we saw earlier can be accessed by
# selecting for the condition == stochasticity_level
print("boards:\n", processed_data.boards.sel(conditions = stochasticity_level, games = 0).values)

# and similarly we can access the actions they took - they should match with what we saw above
print("choose left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values)

# and also the paths taken
print("paths:", processed_data.paths.sel(conditions = stochasticity_level, games = 0).values.tolist())

boards:
 [[[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [6 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 4 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[0 0 0 0 0 0 0 0]

`pov_array` basically takes the "perspective" of the player. You can see for yourself that it matches the path the participant actually took and basically just makes `[0, 0]` the place where the participant "currently" is.

In [133]:
print("pov_array:\n", processed_data.pov_array.sel(conditions = stochasticity_level, games = 0).values)

pov_array:
 [[[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [7 1 5 1 0 0 0 0]
  [2 2 4 2 7 0 0 0]
  [2 3 9 8 5 6 0 0]
  [3 7 9 2 5 4 1 0]
  [9 4 3 3 9 1 3 1]]

 [[4 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [7 1 5 0 0 0 0 0]
  [2 2 4 2 0 0 0 0]
  [2 3 9 8 5 0 0 0]
  [3 7 9 2 5 4 0 0]
  [9 4 3 3 9 1 3 0]
  [0 0 0 0 0 0 0 0]]

 [[6 0 0 0 0 0 0 0]
  [7 1 0 0 0 0 0 0]
  [2 2 4 0 0 0 0 0]
  [2 3 9 8 0 0 0 0]
  [3 7 9 2 5 0 0 0]
  [9 4 3 3 9 1 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[7 0 0 0 0 0 0 0]
  [2 2 0 0 0 0 0 0]
  [2 3 9 0 0 0 0 0]
  [3 7 9 2 0 0 0 0]
  [9 4 3 3 9 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[2 0 0 0 0 0 0 0]
  [3 9 0 0 0 0 0 0]
  [7 9 2 0 0 0 0 0]
  [4 3 3 9 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[4 0 0 0 0 0 0 0]
  [9 2 0 0 0 0 0 0]
  [3 3 9 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[9 0 0 0 0 0 0

## Step 2: Model Setup

In [ ]:
from modeling import Model, filter_depth, value_path

The main thing to know here is that the model will take some parameters `params` and the boards `pov_array` and then make a prediction about the probability the player moves left at each step.

In [199]:
params = {
    "lapse": 0, 
    "condition_inv_temp_0": 10,
    "condition_inv_temp_1": 10,
    "condition_inv_temp_2": 10,
    "condition_inv_temp_3": 10,
    "condition_inv_temp_4": 10,
}
model = Model("policy_compress", filter_fn = filter_depth, value_fn = value_path, variant = variant)

In [200]:
# filter depth filters all the rows beyond row + depth
filtered_pov_array = model.filter_fn(processed_data.pov_array).sel(filter_params = 2, conditions = stochasticity_level, games = 0).transpose("trials", "rows",  "cols")
print(filtered_pov_array.values)

[[[0 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [4 4 2 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[4 0 0 0 0 0 0 0]
  [4 4 0 0 0 0 0 0]
  [7 1 5 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[6 0 0 0 0 0 0 0]
  [7 1 0 0 0 0 0 0]
  [2 2 4 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[7 0 0 0 0 0 0 0]
  [2 2 0 0 0 0 0 0]
  [2 3 9 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[2 0 0 0 0 0 0 0]
  [3 9 0 0 0 0 0 0]
  [7 9 2 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[4 0 0 0 0 0 0 0]
  [9 2 0 0 0 0 0 0]
  [3 3 9 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[9 0 0 0 0 0 0 0]
  [3 9 0

In [201]:
# tells us the value left - value right at each step
model.value_fn(filtered_pov_array)

<xarray.DataArray (trials: 7)> Size: 56B
array([ 0.,  2.,  4., -6., -6.,  1., -6.])
Coordinates:
    conditions     float64 8B 0.5
    filter_params  int64 8B 2
Dimensions without coordinates: trials

In [ ]:
# tells us the probability of moving left at each step
p_left = model.get_prob_left(params, model.filter_fn(processed_data.pov_array))
for depth in range(8): 
    print(f"Predicted p_left (for depth {depth}): ", np.round(p_left.sel(conditions = stochasticity_level, games = 0, filter_params = depth).values, 2))
print("Actual p_left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values.astype(float))

Predicted p_left (for depth 0):  [0.5 0.5 0.5 0.5 0.5 0.5 0.5]
Predicted p_left (for depth 1):  [0.5 0.5 1.  0.5 0.  1.  0. ]
Predicted p_left (for depth 2):  [0.5 1.  1.  0.  0.  1.  0. ]
Predicted p_left (for depth 3):  [1.  0.5 1.  0.  0.  1.  0. ]
Predicted p_left (for depth 4):  [0.5 0.5 1.  0.  0.  1.  0. ]
Predicted p_left (for depth 5):  [0.5 0.5 1.  0.  0.  1.  0. ]
Predicted p_left (for depth 6):  [0.5 0.  1.  0.  0.  1.  0. ]
Predicted p_left (for depth 7):  [0.5 0.  1.  0.  0.  1.  0. ]
Actual p_left:  [1. 1. 1. 0. 0. 0. 0.]


/Users/jordanlei/miniconda3/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)


Let's show an example with maybe more realistic parameters (the previous ones are helpful for illustration but lead to extreme log likelihoods, like -inf): 

In [ ]:
params = {
    "lapse": 0, 
    "condition_inv_temp_0": -1,
    "condition_inv_temp_1": -1,
    "condition_inv_temp_2": -1,
    "condition_inv_temp_3": -1,
    "condition_inv_temp_4": -1,
}
model = Model("policy_compress", filter_fn = filter_depth, value_fn = value_path, variant = variant)
filtered_pov_array = model.filter_fn(processed_data.pov_array)
p_left = model.get_prob_left(params, filtered_pov_array)
print("Predicted p_left: ", np.round(p_left.sel(conditions = stochasticity_level, games = 0).values, 2))
print("Actual p_left: ", processed_data.choose_left.sel(conditions = stochasticity_level, games = 0).values.astype(float))
log_likelihood = processed_data.choose_left * np.log(p_left) + (1 - processed_data.choose_left) * np.log(1 - p_left)
ll = log_likelihood.sum(["games", "trials", "conditions"])

for depth in range(8): 
    print(f"Predicted LL (for depth {depth}): ", ll.sel(filter_params = depth).values)

Predicted p_left:  [[0.5  0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.9  0.5  0.1  0.93 0.1 ]
 [0.5  0.68 0.81 0.1  0.1  0.59 0.1 ]
 [0.68 0.5  0.81 0.1  0.1  0.59 0.1 ]
 [0.5  0.5  0.81 0.1  0.1  0.59 0.1 ]
 [0.5  0.5  0.75 0.1  0.1  0.59 0.1 ]
 [0.5  0.41 0.75 0.1  0.1  0.59 0.1 ]
 [0.5  0.41 0.75 0.1  0.1  0.59 0.1 ]]
Actual p_left:  [1. 1. 1. 0. 0. 0. 0.]
Predicted LL (for depth 0):  -727.8045395879425
Predicted LL (for depth 1):  -847.6490180236201
Predicted LL (for depth 2):  -903.7006738187009
Predicted LL (for depth 3):  -921.6980542994199
Predicted LL (for depth 4):  -935.8162986346367
Predicted LL (for depth 5):  -943.4517383454772
Predicted LL (for depth 6):  -942.001241482742
Predicted LL (for depth 7):  -941.0191262510848


The maximum likelihood function finds the best filter parameter (or parameters, if they are conditional) given a set of parameters

In [ ]:
ml, params = model.select_best_filter_params(params, model.filter_fn(processed_data.pov_array), processed_data.choose_left)
print("Maximum likelihood parameters: ", params)
print("Maximum likelihood log likelihood: ", ml.item())

Maximum likelihood parameters:  {'lapse': 0, 'condition_inv_temp_0': -1, 'condition_inv_temp_1': -1, 'condition_inv_temp_2': -1, 'condition_inv_temp_3': -1, 'condition_inv_temp_4': -1, 'filter_params': {'global': np.int64(0)}}
Maximum likelihood log likelihood:  -727.8045395879425
